In [1]:
import gzip

import pandas as pd

pd.set_option("display.max_columns", None)

Read LOH positions

In [2]:
with open("/home/junkhann/daten/mmml_onek1k_all_patients_columns.txt", mode="r") as columns_file:
    columns_list = columns_file.readlines()

columns_list = [col.rstrip("\n") for col in columns_list]

columns_list[1] = "pos"
columns_list

['#CHROM',
 'pos',
 'position_dummy',
 'REF',
 'ALT',
 'INFO',
 'sample_control',
 'sample_tumor',
 'start',
 'end',
 'genotype',
 'TCN',
 'PID',
 'normal_genotype',
 'tumor_genotype',
 'quality_score',
 'reads_normal',
 'reads_tumor',
 '#CHR',
 'POS',
 'POS_DUMMY',
 'CELL_TYPE',
 'RSID',
 'GENE',
 'GENE_ID',
 'A1',
 'A2',
 'A2_FREQ_ONEK1K',
 'A2_FREQ_HRC']

In [3]:
df = pd.read_csv("/home/junkhann/daten/LOH_positions.vcf.gz", sep="\t", compression="gzip", header=None, names=columns_list, low_memory=False)
df = df.drop(columns=["position_dummy", "POS_DUMMY", "#CHR", "POS"])
len(df)

3210733

In [4]:
df.head()

,#CHROM,pos,REF,ALT,INFO,sample_control,sample_tumor,start,end,genotype,TCN,PID,normal_genotype,tumor_genotype,quality_score,reads_normal,reads_tumor,CELL_TYPE,RSID,GENE,GENE_ID,A1,A2,A2_FREQ_ONEK1K,A2_FREQ_HRC
0,3,116729229,G,A,BRF=0.21;FR=0.25;HP=2;HapScore=1;MGOF=36;MMLQ=...,"1/0:-43.07,0,-50.77:4:99:31:14","0/0:0,-6.02,-62.3:36:60:20:0",116664052,116765897,1:0,1.0,4100314,"1,1","1,0",99.000000,31,20,CD4 Naive/Central memory T cell,rs4855891,RP11-384F7.1,ENSG00000243276,G,A,0.55131,0.537927
1,3,116729818,C,T,BRF=0.12;FR=0.25;HP=1;HapScore=1;MGOF=11;MMLQ=...,"0/1:-76.15,0,-67.55:7:99:54:27","0/0:0,-8.43,-96.7:11:84:30:0",116664052,116765897,1:0,1.0,4100314,"1,1","1,0",99.000000,54,30,CD4 Naive/Central memory T cell,rs4855909,RP11-384F7.1,ENSG00000243276,C,T,0.54896,0.523437
2,3,116737017,T,G,BRF=0.2;FR=0.25;HP=1;HapScore=1;MGOF=19;MMLQ=3...,"1/0:-55.47,0,-39.37:19:99:35:18","0/0:0,-5.72,-69.5:12:57:20:0",116664052,116765897,1:0,1.0,4100314,"1,1","1,0",99.000000,35,20,CD4 Naive/Central memory T cell,rs2868485,RP11-384F7.1,ENSG00000243276,T,G,0.72818,0.715353
3,3,116753833,C,A,BRF=0.17;FR=0.25;HP=1;HapScore=1;MGOF=18;MMLQ=...,"1/0:-45.56,0,-73.06:2:99:39:15","0/0:0,-6.32,-71.4:18:63:24:0",116664052,116765897,1:0,1.0,4100314,"1,1","1,0",99.000000,39,24,CD4 Naive/Central memory T cell,rs651941,RP11-384F7.1,ENSG00000243276,C,A,0.33303,0.327702
4,4,189927806,A,G,BRF=0.29;FR=0.2501;HP=3;HapScore=2;MGOF=12;MML...,"0/1:-13.92,0,-50.02:12:99:25:8","0/0:0,-2.8,-33.9:8:28:10:0",189519987,190939997,1:0,1.0,4100314,"1,1","1,0",51.557951,25,10,Naïve/Immature B Cell,rs4863078,FRG1,ENSG00000109536,A,G,0.12902,0.144780


In [5]:
df["position_code"] = df["#CHROM"].astype("str") + "-" + df["pos"].astype("str")

In [7]:
no_distinct_positions = df["position_code"].nunique()
no_distinct_positions

225485

In [8]:
df["#CHROM"].nunique()

22

In [9]:
no_distinct_patients = df["PID"].nunique()
no_distinct_patients

209

In [10]:
unique_combinations = len(df.drop_duplicates(['PID','position_code']).index)
unique_combinations

237932

Aggregate CELL_TYPE, GENE and GENE_ID

In [11]:
df.columns

Index(['#CHROM', 'pos', 'REF', 'ALT', 'INFO', 'sample_control', 'sample_tumor',
       'start', 'end', 'genotype', 'TCN', 'PID', 'normal_genotype',
       'tumor_genotype', 'quality_score', 'reads_normal', 'reads_tumor',
       'CELL_TYPE', 'RSID', 'GENE', 'GENE_ID', 'A1', 'A2', 'A2_FREQ_ONEK1K',
       'A2_FREQ_HRC', 'position_code'],
      dtype='object')

In [12]:
df_unique = df.groupby(["#CHROM", "pos", "PID"]).agg({'#CHROM': "first", 'pos': "first", 'REF': "first", 'ALT': "first", 'INFO': "first", 'sample_control': "first", 'sample_tumor': "first",
       'start': "first", 'end': "first", 'genotype': "first", 'TCN': "first", 'PID': "first", 'normal_genotype': "first",
       'tumor_genotype': "first", 'quality_score': "first", 'reads_normal': "first", 'reads_tumor': "first",
       'CELL_TYPE': lambda x: ','.join(x), 'RSID': "first", 'GENE': lambda x: set(x), 'GENE_ID': lambda x: set(x), 'A1': "first", 'A2': "first", 'A2_FREQ_ONEK1K': "first",
       'A2_FREQ_HRC': "first", 'position_code': "first"}).reset_index(drop=True)
len(df_unique)

237932

In [13]:
df_unique.head()

,#CHROM,pos,REF,ALT,INFO,sample_control,sample_tumor,start,end,genotype,TCN,PID,normal_genotype,tumor_genotype,quality_score,reads_normal,reads_tumor,CELL_TYPE,RSID,GENE,GENE_ID,A1,A2,A2_FREQ_ONEK1K,A2_FREQ_HRC,position_code
0,1,842013,T,G,BRF=0.19;FR=0.25;HP=4;HapScore=1;MGOF=21;MMLQ=...,"1/0:-36.89,0,-58.19:21:99:33:13","0/0:0,-5.07,-54.5:0:51:18:0",840009,12839977,2:0,2.0,4134434,"1,1","2,0",99.000000,33,18,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs7419119,{UBE2J2},{ENSG00000160087},T,G,0.21688,0.212812,1-842013
1,1,846398,G,A,BRF=0.44;FR=0.25;HP=3;HapScore=1;MGOF=29;MMLQ=...,"1/0:-23.69,0,-17.99:29:99:15:8","0/0:0,-5.12,-66.2:21:51:18:0",840009,12839977,2:0,2.0,4183136,"1,1","2,0",60.703590,15,18,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs58781670,{UBE2J2},{ENSG00000160087},G,A,0.20774,0.216970,1-846398
2,1,848445,G,A,BRF=0.49;FR=0.254;HP=2;HapScore=3;MGOF=3;MMLQ=...,"1/0:-20.1,0,-34.3:3:99:15:5","0/0:0,-1.56,-51.1:2:16:16:0",840009,12839977,2:0,2.0,4183136,"1,1","2,0",53.958747,15,16,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs4626817,{UBE2J2},{ENSG00000160087},G,A,0.20797,0.210548,1-848445
3,1,850780,C,T,BRF=0.17;FR=0.25;HP=1;HapScore=1;MGOF=4;MMLQ=2...,"0/1:-55.58,0,-49.58:4:99:46:23","0/0:0,-6.32,-76.5:1:63:25:0",840009,12839977,2:0,2.0,4149246,"1,1","2,0",99.000000,46,25,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs6657440,{UBE2J2},{ENSG00000160087},C,T,0.60181,0.602125,1-850780
4,1,851190,G,A,BRF=0.29;FR=0.2501;HP=1;HapScore=1;MGOF=4;MMLQ...,"1/0:-42.59,0,-62.89:2:99:30:12","0/0:0,-3.12,-73.7:4:31:20:0",840009,12839977,2:0,2.0,4176325,"1,1","2,0",86.359655,30,20,"Naïve/Immature B Cell,Memory B Cell,CD4 Effect...",rs28609852,{UBE2J2},{ENSG00000160087},G,A,0.21290,0.213474,1-851190


Check genotype fields

In [14]:
df_unique["normal_genotype"].value_counts()

normal_genotype
1,1    237932
Name: count, dtype: int64

In [15]:
df_unique["tumor_genotype"].value_counts()

tumor_genotype
1,0    108535
2,0    105829
4,0     15893
3,0      5761
5,0       750
0,1       613
6,0       508
0,2        37
0,3         4
8,0         1
0,4         1
Name: count, dtype: int64

Check equality of REF - ALT and A1 - A2

In [16]:
df_unique["_equal"] = (df_unique["REF"] == df_unique["A1"]) & (df_unique["ALT"] == df_unique["A2"])

In [17]:
df_unique["_equal"].value_counts()

_equal
True     235331
False      2601
Name: count, dtype: int64

In [18]:
df_unique = df_unique[df_unique["_equal"] == True]
df_unique = df_unique.drop(columns="_equal")
len(df_unique)

235331

Check number of different ALT alleles per position

In [19]:
def count_unique_alt_alleles(df_pos: pd.DataFrame, d: dict):
    alt_alleles = df_pos["ALT"].nunique()
    pos = df_pos.name
    d[pos] = alt_alleles

In [20]:
count_dict = {}
df_unique.groupby("position_code").apply(count_unique_alt_alleles, count_dict) 

""


In [21]:
set(count_dict.values())

{1}

Write to file

In [22]:
df_unique.to_csv(f"/home/junkhann/daten/LOH_positions_aggregated.tsv.gz", sep="\t", header=True, index=False, compression="gzip")